In [1]:
from cmath import pi, rect
from itertools import combinations, product
from math import prod
import pygraphviz as pgv


n = 3

def f(x):
    assert all(t in [0, 1, 2] for t in x)
    return 1 if sum(x) % 3 == 0 else -1 # mod3
    # return prod([(1 if t == 0 else -1) for t in x]) # xor


def dot(x, v):
    assert all(t in [0, 1, 2] for t in x) and all(t in [0, 1, 2] for t in v) and len(x) == len(v)
    return sum([x[i] * v[i] for i in range(len(x))]) % 3


def fourier_coefficient(v):
    return sum(f(x) * rect(1, 2 * pi * dot(x, v) / 3) for x in product([0, 1, 2], repeat=len(v))) / 3 ** len(v)


coefficients = dict()
for v in product([0, 1, 2], repeat=n):
    coefficients[v] = fourier_coefficient(v)
    if abs(coefficients[v]) > 0.00001:
        print(f'{v} {fourier_coefficient(v):.2f}')

strings = [''.join(a) for a in product('012', repeat=n)]

graph = pgv.AGraph(directed=True, rankdir='BT')
for s1, s2 in combinations(strings, 2):
    diff = [i for i in range(n) if s1[i] != s2[i] and (s1[i] == '0' or s2[i] == '0')]
    if len(diff) == 1:
        i = diff[0]
        if s1[i] != '0':
            graph.add_edge(s2, s1)
            graph.get_edge(s2, s1).attr['color'] = 'white'
        else:
            graph.add_edge(s1, s2)
            graph.get_edge(s1, s2).attr['color'] = 'white'

for v in graph.nodes():
    graph.get_node(v).attr['style'] = 'filled'
    int_v = tuple([int(s) for s in v])
    coef = coefficients[int_v]
    graph.get_node(v).attr['fillcolor'] = 'grey' + str(int((1 - abs(coef)) * 99))

graph.layout(prog='dot')
graph.draw('fourier_coefficients.png')

(0, 0, 0) -0.33+0.00j
(1, 1, 1) 0.67-0.00j
(2, 2, 2) 0.67-0.00j


![coefficient](fourier_coefficients.png)